In [17]:
import ee
import geemap

In [18]:
ee.Initialize()

In [19]:
aoi = ee.FeatureCollection("projects/nb-lidar/assets/aoi_nb_south")
features = ee.FeatureCollection("projects/nb-lidar/assets/nb_south_training")

In [20]:
from src.helpers.image_processing import process_and_stack_images

stack = process_and_stack_images(aoi, "dtm")

In [21]:
# pre processing
label_col = "class_name"
labels = features.aggregate_array(label_col).distinct()
new_lables = ee.List.sequence(1, labels.size())

features = features.remap(labels, new_lables, label_col)

In [23]:
# extract features
features = stack.sampleRegions(
    collection=features,
    scale=10,
    tileScale=16,
    geometries=True,
).randomColumn()

In [24]:
import ee.batch
import time

feature_task = ee.batch.Export.table.toAsset(
    collection=features, assetId="projects/nb-lidar/assets/dtm_features"
)

feature_task.start()

while feature_task.active():
    print("Polling......")
    time.sleep(10)
print("Task Complete..")

Polling......
Polling......
Polling......
Polling......
Polling......
Polling......
Polling......
Polling......
Polling......
Polling......
Polling......
Polling......
Polling......
Polling......
Polling......
Polling......
Polling......
Polling......
Task Complete..


In [9]:
from src.helpers.classifier import SmileRandomForest

rf = SmileRandomForest(100)
rf.train(train, label_col, stack.bandNames())

prediction = rf.predict(stack)

In [10]:
error = rf.error_matrix(test)
error

In [ ]:
# Map = geemap.Map()

# Map.addLayer(prediction, {'min': 1, 'max': 8})

# Map.centerObject(aoi, 10)

# Map

KeyboardInterrupt: 